# Arduino IOT

This notebook describes a simple Arduino project that streams data to an Azure IOT Hub via WiFi. As a backup plan (always have a backup) we also introduce the Particle Photon (company and device respectively) with considerably more computing power than the Arduino; but a newer technology not as well supported.

Arduino is an umbrella term (company / project / community) for open hardware (embedded) devices and associated software that is extensively used in the Maker community. If you are unfamiliar with Arduino please begin by reading through https://en.wikipedia.org/wiki/Arduino. YouTube videos will also prove useful!

The aim here is to build an Internet Of Things (IOT) framework that combines the Arduino leaf stack with the the Azure IOT Hub cloud stack. To cut to the chase I provide some summary bullets from the perspective of 'how do I get stuff to work?'

* The most commonly used Arduino form factor is the Arduino Uno ($15).  It includes a set of headers (wire connection points) that facilitate _rapid prototyping_. 
* To get a project off and running requires the following: 
    * An Arduino (Arduino Uno in this case)
    * Probably some peripheral hardware devices and basic parts
    * Connecting wires and a breadboard
    * A 6V power supply and barrel power connector
    * A computer
        * Need the Arduino IDE installed
        * Need a USB cable for programming the Arduino: Also provides power when plugged in
* To build a project into a ruggedized assembly requires some additional thought and time
    * Enclosure
    * Arduino 'project shields' provide solder boards to replace jamming wires into headers!
    
* We have three problems to address at the outset:
    * Configure a sensor assembly
    * Telemeter data to the cloud
    * Keep that data organized and monitored
* In principle we have three corresponding answers at the outset
    * The Arduino stack facilitates the process of configuring sensors-to-data
    * WiFi (or phone-based) data connectivity is also available within Arduino
    * Azure (among other public cloud providers) has built IOT hub to help manage data

This notebook takes these three topics in sequence.

The basic hardware is an Arduino Uno with the following associated devices: 

* A standard servo motor for the sake of an actuation test
    * The Arduino can rotate the servo head for example in response to an external directive
* A WiFi shield
* An I2C OLED module to act as a local diagnostic printout
    * OLED = Organic LED... so in other words it is a little pixelated screen
    * I2C abbreviates IIC = Inter-integrated circuit serial bus (communication system)
    * This module: Diymall® 0.96" Inch Yellow and Blue 128x64 OLED
    * See http://www.amazon.com/dp/B00O2LLT30?psc=1

### OLED Display Instructions

Based on Arduino IDE and UNO R3 compatible board: In the Arduino app: 

* Go to menu option "Sketch > Include Library > Manage Libraries" and install "Adafruit GFX Library" and "Adafruit SSD1306". This will get the appropriate libraries installed.
* Go to menu option "File > Examples > Adafruit SSD1306 > ssd1306_128x64_i2c" to bring up example code 
* Find the line "display.begin(SSD1306..." and change the address from 0x3D to 0x3C
    * The jumper I2C addresses on the board are mis-labeled 0x78 etc
* Pinout
    * OLED <> UNO
    * VCC <> 5V 
    * GND <> GND
    * SDA <> A4
    * SCL <> A5 
* This display is split upper yellow and the lower section is a teal blue
* I found that you DID NOT have to adjust the header file and only need to initialize the display with 

display.begin(SSD1306_SWITCHCAPVCC, 0x3C);

    * Unnecessary alternative: Library .h file lines 54--56 #defines: 
        * Leave the one that has 128 x 64 uncommented and comment out the other display sizes. 

Worked on both an Uno and a Mega. The example code for the SSD1306 really starts to push the memory of the Uno, but the Mega has no issue loading the program. Because it is an OLED, it will look dead until it has been initialized, which can be a bit confusing when you are first testing the screen. I thought I had a dud until I got it correctly initialized. Had no issues with 3.3v or 5v.

I scanned the I2C bus using a sketch on github: "https://github.com/todbot/arduino-i2c-scanner/blob/master/I2CScanner/I2CScanner.ino". 
It showed "addr: 60 found!" Convert 60 to hex and you get 0x3C. Good luck!
Top 16 rows (0-15) are Yellow, then Next 48 are blue. 
Glass can be a bit fragile, so if you mount the display on something (like plexiglas), don't go too tight. 
If you want to display bitmap images, then search google for "lcd-image-converter" windows app. 
It will allow you to import a monochrome .BMP image and output a static char array that can be used in 
your c/c++/arduino code and passed to the ssd1306 library display.drawBitmap() function. Once you have 
a monochrome image, using lcd-image-converter, import your image, then under options, use the "monochrome" 
preset, type=monochrome/diffuse dither, main scan direction=top to bottom, line scan=fwd, image block size=8 
bit, then click "show preview" and copy this code into your static array following the adafruit 
ssd1306_128x64_i2c.ino/ testdrawbitmap() function example. Lastly, OLEDs are susceptible to "screen burn-in". 
Use a screensaver or blank your screen on occasion -- that is, if you care. 

In [ ]:
// Tested with a 30k Thermistor.

#include <SPI.h>
#include <Wire.h>
#include <Adafruit_GFX.h>
#include <Adafruit_SSD1306.h>

#define OLED_RESET 4

Adafruit_SSD1306 display(OLED_RESET);

#if (SSD1306_LCDHEIGHT != 64)
#error("Height incorrect, please fix Adafruit_SSD1306.h!");
#endif

#define THERMISTORPIN A0 // which analog pin to connect
#define THERMISTORNOMINAL 30000 // resistance at 25 degrees C
#define TEMPERATURENOMINAL 25 // temp. for nominal resistance (almost always 25 C)
#define NUMSAMPLES 50 // how many samples to take and average, more takes longer
#define BCOEFFICIENT 4400 // The beta coefficient of the thermistor (usually 3000-4000)
#define SERIESRESISTOR 30000 // the value of the 'other' resistor

int samples[NUMSAMPLES];

void setup() {
 Serial.begin(9600);

 // connect AREF to 3.3V and use that as VCC, less noisy!
 // connect AREF to 3.3V and use that as VCC, less noisy!
 analogReference(EXTERNAL);

 // by default, we'll generate the high voltage from the 3.3v line internally! (neat!)
 display.begin(SSD1306_SWITCHCAPVCC, 0x3C); // initialize with the I2C addr 0x3D (for the 128x64)
 // init done

 // Show image buffer on the display hardware.
 // Since the buffer is intialized with an Adafruit splashscreen
 // internally, this will display the splashscreen.
 display.display();
 delay(250);

 // Clear the buffer.
 display.clearDisplay();
}

void loop() {

 uint8_t i;
 float average;
 float maxTemp;
 maxTemp = 0;
 // take N samples in a row, with a slight delay
 for (i=0; i< NUMSAMPLES; i++) {
 samples[i] = analogRead(THERMISTORPIN);
 delay(10);
 }

 // average all the samples out
 average = 0;
 for (i=0; i< NUMSAMPLES; i++) {
 average += samples[i];
 }
 average /= NUMSAMPLES;

 display.clearDisplay();
 display.setTextSize(1);
 display.setTextColor(WHITE);
 display.setCursor(0,0);

 Serial.print("Analog ");
 Serial.println(average);
 display.print("Analog ");
 display.println(average);

 // convert the value to resistance
 average = 1023 / average - 1;
 average = SERIESRESISTOR / average;
 Serial.print("R ");
 Serial.println(average);
 display.print("R-Val ");
 display.println(average);
 display.setTextSize(2);

 float steinhart;
 float fconvert;
 float convertValue;
 steinhart = average / THERMISTORNOMINAL; // (R/Ro)2
 steinhart = log(steinhart); // ln(R/Ro)
 steinhart /= BCOEFFICIENT; // 1/B * ln(R/Ro)
 steinhart += 1.0 / (TEMPERATURENOMINAL + 273.15); // + (1/To)
 steinhart = 1.0 / steinhart; // Invert
 steinhart -= 273.15; // convert to C
 Serial.print(steinhart);
 Serial.println(" *C");
 fconvert = steinhart * 1.8;
 fconvert = fconvert + 32;
 Serial.print("Temperature ");
 display.println("A0 Temp");
 Serial.print(fconvert);
 Serial.println(" *F");
 display.print(steinhart);
 display.println(" *C");
 display.print(fconvert);
 display.println(" *F");
 display.display();

 delay(250);
}